In [23]:
from bokeh.layouts import column
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook
import numpy as np
output_notebook()

Loading BokehJS ...

In [2]:
import serial

port = serial.Serial("/dev/ttyACM0", 115200)

In [24]:
source = ColumnDataSource(data=dict(a_x=[], a_y=[], a_z=[],
                                    g_x=[], g_y=[], g_z=[],
                                    i=[], dt=[], temp=[]))

p1 = figure(width=600, height=300, title="Accelerations")
p1.line(x='i', y='a_x', source=source)
p1.line(x='i', y='a_y', source=source)
p1.line(x='i', y='a_z', source=source)

p2 = figure(width=600, height=300, title="Gyroscope")
p2.line(x='i', y='g_x', source=source)
p2.line(x='i', y='g_y', source=source)
p2.line(x='i', y='g_z', source=source)

bokeh_handle = show(column(p1, p2), notebook_handle=True)


def update_plot(new_data):
    source.stream(new_data)
    push_notebook(handle=bokeh_handle)

In [29]:
labels = "dt,i,a_x,a_y,a_z,g_x,g_y,g_z,temp".split(',')
vals = sample(range(-2**16+1, 2**16), len(labels))
new_data = dict(zip(labels, [[val] for val in vals]))
update_plot(new_data)

In [8]:
def serial_readout_loop(port: Serial):
    labels = "dt,i,a_x,a_y,a_z,g_x,g_y,g_z,temp".split(',')
    #line = port.readline()
    for line in port:
        deco = line.decode().rstrip()
        new_data = dict(zip(labels, [[int(val)] for val in deco.split(',')]))
        update_plot(new_data)
    #return new_data

In [9]:
mock_serial_readout_loop(port)

{'dt': [4999],
 'i': [286577],
 'a_x': [56],
 'a_y': [-1700],
 'a_z': [18856],
 'g_x': [149],
 'g_y': [165],
 'g_z': [-126],
 'temp': [2704]}

In [3]:
from random import sample
from time import sleep
import threading as th

def mock_serial_readout_loop(port):
    labels = "dt,i,a_x,a_y,a_z,g_x,g_y,g_z,temp".split(',')
    
    while True:
        vals = sample(range(-2**16+1, 2**16), len(labels))
        new_data = dict(zip(labels, [[val] for val in vals]))
        update_plot(new_data)
        sleep(0.5)

loop = th.Thread(target=mock_serial_readout_loop, args=(None,))
loop.start()
loop.join()

KeyboardInterrupt: 

Exception in thread Exception in threading.excepthook:
Exception ignored in thread started by: <bound method Thread._bootstrap of <Thread(Thread-5 (mock_serial_readout_loop), stopped 126730806228544)>>
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1030, in _bootstrap
    self._bootstrap_inner()
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self._invoke_excepthook(self)
  File "/usr/lib/python3.12/threading.py", line 1389, in invoke_excepthook
    local_print("Exception in threading.excepthook:",
  File "/home/being/.virtualenvs/tests/lib/python3.12/site-packages/ipykernel/iostream.py", line 604, in flush
    self.pub_thread.schedule(self._flush)
  File "/home/being/.virtualenvs/tests/lib/python3.12/site-packages/ipykernel/iostream.py", line 267, in schedule
    self._event_pipe.send(b"")
  File "/home/being/.virtualenvs/tests/lib/python3.12/site-packages/zmq/sugar/socket.py", line 701, in send
    return super().send(